In [1]:
#save model example
import tensorflow as tf
v1=tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="v2")
result=v1+v2

init_op=tf.global_variables_initializer()
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess,"/home/lilin/model/model.ckpt")

In [2]:
#another version to restore
import tensorflow as tf

with tf.Session() as sess:
    saver=tf.train.import_meta_graph("/home/lilin/model/model.ckpt.meta")
    saver.restore(sess,"/home/lilin/model/model.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))
    

INFO:tensorflow:Restoring parameters from /home/lilin/model/model.ckpt
[ 3.]


In [ ]:
#one way to restore the model
import tensorflow as tf
v1=tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="v2")

result=v1+v2

saver=tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess,"/home/lilin/model/model.ckpt")
    print(sess.run(result))
    

In [ ]:
#rename a variable
v1=tf.Variable(tf.constant(1.0,shape=[1]),name="name=other-v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="name=other-v2")

saver=tf.train.Saver({"v1":v1,"v2":v2})


# another example about slidemean

In [ ]:
import tensorflow as tf

v=tf.Variable(0,dtype=tf.float32,name="v")

for variables in tf.all_variables():
    print(variables.name)
    
ema=tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op=ema.apply(tf.all_variables())

for variables in tf.all_variables():
    print(variables.name)
    
saver=tf.train.Saver()
with tf.Session() as sess:
    init_op=tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v,10))
    sess.run(maintain_averages_op)
    saver.save(sess,"/home/lilin/model/model_01/model.ckpt")
    print(sess.run([v,ema.average(v)]))
    #expect out is:[10.0,0.99999905]

In [ ]:
v=tf.Variable(0,dtype=tf.float32,name="v")

saver=tf.train.Saver({"v/ExponentialMovingAverage":v})

with tf.Session() as sess:
    saver.restore(sess,"/home/lilin/model_01/model.ckpt.meta")
    print(sess.run(v))
    #expect out is:0.99999905

In [ ]:
import tensorflow as tf
v=tf.Variable(0,dtype=tf.float32,name="v")
ema=tf.train.ExponentialMovingAverage(0.99)

print(ema.variables_to_restore())

saver=tf.train.Saver(ema.variables_to_restore())

with tf.Session() as sess:
    saver.restore(sess,"/home/lilin/model_01/model.ckpt")
    print(sess.run(v))
    
    

In [ ]:
#
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1=tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="v2")

result=v1+v2

init_op=tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init_op)
    
    graph_def=tf.get_default_graph().as_graph_def()
    
    output_graph_def=graph_util.convert_variables_to_constants(sess,graph_def,['ass'])
    
    with tf.gfile.GFile("/home/lilin/model_01/combined_model.pb","wb") as f:
        f.write(out_graph_def.SerializerToString())
        

In [ ]:
#
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_filename="/home/lilin/model/model_01/combined_model.pb"
    
    with gfile.FastGfile(model_filename,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
        result=tf.import_graph_def(graph_def,return_elements=["add:0"])
        print(sess.run(result))